#

# Car Data Monitoring Experiments

This notebook aims at experimenting with the data obtained from the car data recorder.

In [ ]:
from glob import glob
import numpy as np
import polars as pl
import plotly.express as px

In [ ]:
files = sorted(glob("/Volumes/KARTAIL/gps/*.csv"))
dfs = []
for file in files:
    try:
        dfs.append(pl.read_csv(file))
    except pl.exceptions.NoDataError:
        print(f"Skipped empty dataframe: {file}")

df: pl.DataFrame = pl.concat(dfs)

df = df.with_columns(pl.col("datetime").str.to_datetime(strict=False))
df = df.sort(by=["session", "millis"])
df = df.drop_nulls(subset=["datetime"])

#df = df.filter(
#    (pl.col("session") == pl.max("session"))
#    | (pl.col("session") == pl.max("session") - 1)
#)

df

## Time stability

First, let's explore the most important reference variable: datetime and millis. Are they stable?

In the firmware code, there are 3 clocks:
- The internal ESP32 clock, which is independant of external sources but may drift against true time
- The GPS clock, which is synced with satellite clock information internaly, every time it receives a signal.
- A software wrapper, that I coded on top of a Clock library, is synced when the GPS gives updates. It uses the internal ESP32 clock to add millisecond precision.

To ensure that all information is available, I log every event with the synced software clock's timestamp. But I also add the internal ESP32 clock as a backup measure.
Let's see if this clocks are in sync.

In [ ]:
fig = px.scatter(data_frame=df, x='datetime', y='millis', title="Millis() vs time")
fig.show()

Looking at the two timestamps from a trip recording, we can see that
- A few minutes of data are missing. This is due to GPS Signal loss. Not a big deal here, since 1 minute of data will probably be filled thanks to dead reckoning with other sensors and CAN Bus data.
- It looks like there is no significative drift between the two clocks since we have a clean straight line. This is good.

Now let's take a look at the delta time, which will give us a more precise look. There should be the same time variations for each clock.

In [ ]:
df = df.with_columns(
    delta_time_gps=pl.col('datetime').diff(),
    delta_time_millis=pl.duration(milliseconds=(pl.col('millis').diff()))
)

In [ ]:
fig = px.ecdf(
    df.with_columns(
        pl.col("delta_time_gps").cast(pl.Float32) / 1000000,
        pl.col("delta_time_millis").cast(pl.Float32) / 1000000
    ), x=['delta_time_gps', 'delta_time_millis'],
    title='Time variations of the two clocks')
fig.update_layout(xaxis_range=[0.0, 2.0])
fig.show()

Zooming at the data between 0s and 2s, we can see that the clocks are fairly stable around 1s.

In [ ]:
EARTH_RADIUS = 6371000

def haversine_delta_np(dlat, dlon, lat):
    lat = np.radians(lat)
    dlat = np.radians(dlat)
    dlon = np.radians(dlon)

    a = np.sin(dlat/2.0)**2 + np.cos(lat) * np.cos(lat + dlat) * np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return EARTH_RADIUS * c

def distance_3d_from_deltas(dlat, dlon, dalt, lat):
    surface = haversine_delta_np(dlat, dlon, lat)
    return np.sqrt(surface**2 + dalt**2)

In [ ]:
df = df.with_columns([
    pl.col("latitude").diff().alias("delta_lat"),
    pl.col("longitude").diff().alias("delta_lon"),
    pl.col("altitude").diff().alias("delta_alt"),
]).drop_nulls(subset=["delta_lat"])

df = df.with_columns([
    pl.struct(["delta_lat", "delta_lon", "delta_alt", "latitude"]).map_elements(
        lambda x: distance_3d_from_deltas(
            x["delta_lat"], x["delta_lon"], x["delta_alt"], x["latitude"]
        ),
        return_dtype=pl.Float64
    ).alias("distance_meters"),
])

df

In [ ]:
df_tmp = df.group_by(
    pl.col("session")
).agg(
    pl.sum("distance_meters").alias("total_distance")
).filter(
    (pl.col("total_distance") < 15_000)
    & (pl.col("total_distance") > 8_000)
)

fig = px.box(
    x=df_tmp["total_distance"],
)

fig.show()

In [ ]:
df_tmp

In [ ]:
df_test = pl.read_csv("/Volumes/KARTAIL/imu/32.csv")
df_test = df_test.with_columns(
    (pl.col("millis") * 1_000_000).cast(pl.Time())
)

In [ ]:
df_test

In [ ]:
df_tmp = df.filter(
    pl.col("session") == 32
)
df_tmp = df_tmp.with_columns(
    (pl.col("millis") * 1_000_000).cast(pl.Time())
)

fig = px.line(
    x=df_tmp["millis"],
    y=df_tmp["altitude"]
)

fig.show()

fig = px.line(
    x=df_test["millis"],
    y=df_test["mx"]
)

fig.show()